In [1]:
# !conda install -c anaconda psycopg2
import sys
print(sys.prefix)
# !conda install --yes --prefix {sys.prefix} psycopg2

/home/alfonso/anaconda3


In [ ]:
import pandas as pd
from configparser import ConfigParser
from sqlalchemy import create_engine
import psycopg2

In [ ]:
!conda install -c anaconda psycopg2


In [ ]:
dt_dbase = {
    "host": "database-1.cwazgjcgbajg.us-east-1.rds.amazonaws.com",
    "port": "4321",
    "database": "factor_emision_FE",
    "user": "postgres",
    "password": "MSALAZAR2020*",
}

In [ ]:
def pg_config(filename='connection.ini', section='pg_fedegan'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}

    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]

    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db


def pg_connection_str(**kwargs):
    """
    Create string connection for using with pandas read_sql
    :param kwargs:
    :return:
    """
    params = pg_config(**kwargs)
    conn = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**params)

    return conn


def pg_connection(**kwargs):
    """ Connect to the PostgreSQL database server """
    conn = None

    try:
        # read connection parameters
        params = pg_config(**kwargs)

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    return conn

In [ ]:
conn = pg_connection_str()

query = """SELECT * FROM public.variedad_pasto WHERE "PC_dieta" > 10"""

df1 = pd.read_sql_table("variedad_pasto", conn, index_col='id_variedad')
df2 = pd.read_sql(query, conn, index_col='id_variedad')

print(df1.head())
print(df2.head())

In [ ]:
users = [
    {'idUser': 1, 'uNombre': 'Alfonso', 'uPassword': 'Alfonso'},
    {'idUser': 2, 'uNombre': 'Mauro', 'uPassword': 'prueba2020'},
    {'idUser': 3, 'uNombre': 'Carlos', 'uPassword': 'Carlos2020'},
]

with pg_connection() as conn_pg:
    for user in users:
        cursor = conn_pg.cursor()
        query = """
        insert into users("idUser", "unombre", "upassword") values({idUser}, '{uNombre}', '{uPassword}')
        on conflict on constraint "idUser" 
        do update set "unombre" = '{uNombre}', "upassword" = '{uPassword}'
        """.format(**user)

        cursor.execute(query)
    
    conn_pg.commit()

In [ ]:
conn_str = pg_connection_str()
table = 'variedad_pasto'

query_fields = """
select column_name 
from information_schema.columns
where table_name = '{table}'
order by ordinal_position 
""".format(table=table)

df = pd.read_sql(query_fields, conn_str)
df_model = pd.DataFrame(columns=df['column_name'])
df_model.to_csv('{}.csv'.format(table), index=False)